In [1]:
import pandas as pd
import numpy as np
import pandasql as pdsql
pysql = lambda q: pdsql.sqldf(q, globals())

In [3]:

#sql to filter relevant stuff only
call_log=pd.read_csv('data/call_log_database.csv')
filter="select CALLING_NO, CALLED_NO, CALL_DATE,DURATION,DIRECTION,RECORD_TYPE,ROAM_IND as IS_ROAMING from call_log ;"
call_log=pysql(filter)
call_log

AttributeError: module 'pandas' has no attribute 'load_csv'

In [40]:
#save a backup
call_log.to_csv('generated/call_log.csv',sep=',')

In [34]:
url='generated/call_log.csv'
call_log = pd.read_csv(url)

In [55]:
#for gettng total incoming call duration
incalls="select CALLED_NO,SUM(DURATION) AS TOT_IN_DUR from call_log where RECORD_TYPE<>'SMS'  GROUP BY CALLED_NO;"
#for gettng total outgoing call duration
outcalls="select CALLING_NO,SUM(DURATION) AS TOT_OUT_DUR from call_log where RECORD_TYPE<>'SMS' GROUP BY CALLING_NO;"
#for gettng total incoming call frequency
incallfreq="select CALLED_NO,COUNT(CALLED_NO) AS TOT_IN_CALLS from call_log where RECORD_TYPE<>'SMS' GROUP BY CALLED_NO;"
#for gettng total outgoing call frequency
outcallfreq="select CALLING_NO,COUNT(CALLING_NO) AS TOT_OUT_CALLS from call_log where RECORD_TYPE<>'SMS'  GROUP BY CALLING_NO;"
#run the queries and get the repective tables
in_calls=pysql(incalls)
out_calls=pysql(outcalls)
in_call_freq=pysql(incallfreq)
out_call_freq=pysql(outcallfreq)

In [56]:
#fuses incoming and outgoing tables separately
intable= "select a.*, b.TOT_IN_CALLS from in_calls a inner join in_call_freq b on a.CALLED_NO = b.CALLED_NO order by TOT_IN_CALLS DESC;"
in_table=pysql(intable)
outtable= "select a.*, b.TOT_OUT_CALLS from out_calls a inner join out_call_freq b on a.CALLING_NO = b.CALLING_NO order by TOT_OUT_CALLS DESC;"
out_table=pysql(outtable)

In [57]:
#save a backup(optional)
in_table.to_csv('generated/in_table.csv',sep=',')
out_table.to_csv('generated/out_table.csv',sep=',')

In [58]:

#this part is for fusing the user based incoming and outgoing records into a single table (full outer join)
#along with tendency to call a person

inset=set(in_table['CALLED_NO'])
outset=set(out_table['CALLING_NO'])
#create new dataframe
columns = ['NUMBER','TOT_IN_CALLS', 'TOT_IN_DUR','TOT_OUT_CALLS', 'TOT_OUT_DUR','TENDENCY']
#tendency= total out duration/total duration
df = pd.DataFrame(columns=columns)
#all unique numbers in database
allnumbers=inset.union(outset)
#fusing in table and out table (outer join)
i=0
for number in allnumbers:
    #print(number)
    num = "\"" + number +"\""
    if number not in outset:
        query="select * from in_table where CALLED_NO="+num+";"
        data=pysql(query)
        #tendency=0
        df.loc[i]=[number,data['TOT_IN_CALLS'][0],data['TOT_IN_DUR'][0],0,0,0]   
    elif number not in inset:
        query="select * from out_table where CALLING_NO="+num+";"
        data=pysql(query)
        df.loc[i]=[number,0,0,data['TOT_OUT_CALLS'][0],data['TOT_OUT_DUR'][0],1]   
    else:
        query="select a.*, b.TOT_OUT_DUR,b.TOT_OUT_CALLS from in_table a inner join out_table b on a.CALLED_NO = b.CALLING_NO where a.CALLED_NO="+num+";"
        data=pysql(query)
        tendency=data['TOT_OUT_DUR'][0]/(data['TOT_IN_DUR'][0]+data['TOT_OUT_DUR'][0])
        df.loc[i]=[number,data['TOT_IN_CALLS'][0],data['TOT_IN_DUR'][0],data['TOT_OUT_CALLS'][0],data['TOT_OUT_DUR'][0],tendency]
    i+=1

In [59]:
#display our table
pysql("select * from df order by TENDENCY DESC")

,NUMBER,TOT_IN_CALLS,TOT_IN_DUR,TOT_OUT_CALLS,TOT_OUT_DUR,TENDENCY
0,9996039787,0,0,3,1026,1
1,9829167468,0,0,1,91,1
2,5692201111,0,0,1,35,1
3,9411290999,0,0,1,142,1
4,9311701656,0,0,2,282,1
5,9410459243,0,0,1,65,1
6,9997268708,0,0,1,124,1
7,9219456550,0,0,1,58,1
8,9761388422,0,0,1,34,1
9,9808780217,0,0,8,1059,1


In [60]:
df.to_csv('generated/user_tendency.csv',sep=',')

In [61]:
len(allnumbers)

6553

In [8]:
#sql to filter relevant stuff only
call_log2=pd.read_csv('data/call_log_database.csv')
filter="select CALLING_NO, CALLED_NO, CALL_DATE,DURATION,DIRECTION,RECORD_TYPE,ROAM_IND as IS_ROAMING from call_log2 where DURATION=1 AND RECORD_TYPE=\"VOICE\" ;"
call_log2=pysql(filter)
call_log2

,CALLING_NO,CALLED_NO,CALL_DATE,DURATION,DIRECTION,RECORD_TYPE,IS_ROAMING
0,9837373649,9045885484,01-11-2011 01:15,1,IN,VOICE,N
1,8439002526,9897226613,01-11-2011 08:13,1,OUT,VOICE,N
2,8439312946,9758660213,01-11-2011 11:11,1,OUT,VOICE,N
3,7417510001,9219694115,01-11-2011 11:51,1,OUT,VOICE,N
4,7417510001,9219694115,01-11-2011 11:51,1,IN,VOICE,N
5,8439312946,9557577723,01-11-2011 14:17,1,OUT,VOICE,N
6,8439312946,9997651342,01-11-2011 16:15,1,OUT,VOICE,N
7,8791519500,8410179682,01-11-2011 17:20,1,OUT,VOICE,Y
8,9548904857,9045177771,01-11-2011 18:34,1,IN,VOICE,N
9,7895912755,9045885484,01-11-2011 20:31,1,IN,VOICE,N
